In [8]:
import os
import datetime as dt
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()

import openai


print(openai.__version__)

1.14.2


In [ ]:
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [11]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

[답변]: content='대한민국의 수도는 서울입니다.' response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 24, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}


### Prompt Template

In [12]:
from langchain.prompts import PromptTemplate

# 질문 템플릿 형식 정의
template = "{country}의 수도는 뭐야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 뭐야?')

In [13]:
from langchain.chains import LLMChain

# 연결된 체인(Chain)객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

llm_chain.invoke({"country": "대한민국"})

{'country': '대한민국', 'text': '대한민국의 수도는 서울이야.'}

In [14]:
llm_chain.invoke({"country": "인도"})

{'country': '인도', 'text': '인도의 수도는 뉴델리입니다.'}

In [16]:
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

response = llm_chain.apply(input_list)
response

[{'text': '호주의 수도는 캔버라입니다.'},
 {'text': '중국의 수도는 베이징(北京)입니다.'},
 {'text': '네덜란드의 수도는 암스테르담입니다.'}]

### generate로 LLMResult 정보 확인

In [17]:
response = llm_chain.generate(input_list)
response

LLMResult(generations=[[ChatGeneration(text='호주의 수도는 캔버라입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='호주의 수도는 캔버라입니다.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 17, 'total_tokens': 30}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}))], [ChatGeneration(text='중국의 수도는 베이징(北京)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='중국의 수도는 베이징(北京)입니다.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 19, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}))], [ChatGeneration(text='네덜란드의 수도는 암스테르담입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='네덜란드의 수도는 암스테르담입니다.', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 22, 'tot

In [18]:
response.llm_output

{'token_usage': {'completion_tokens': 53,
  'prompt_tokens': 58,
  'total_tokens': 111},
 'model_name': 'gpt-3.5-turbo',
 'system_fingerprint': 'fp_3bc1b5746c'}

In [24]:
[a[0].text for a in response.generations]

['호주의 수도는 캔버라입니다.', '중국의 수도는 베이징(北京)입니다.', '네덜란드의 수도는 암스테르담입니다.']

### 2개 이상의 변수

In [26]:
# 질문 템플릿 형식 정의
template = "{area1} 와 {area2} 의 시차는 몇시간이야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template)

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [27]:
input_list = [
    {"area1": "파리", "area2": "뉴욕"},
    {"area1": "서울", "area2": "하와이"},
    {"area1": "켄버라", "area2": "베이징"},
]

# 반복문으로 결과 출력
result = llm_chain.apply(input_list)
for res in result:
    print(res["text"].strip())

파리와 뉴욕의 시차는 6시간입니다. 파리는 그리니치 평균시(GMT+1)를 따르고, 뉴욕은 동부 표준시(EST 또는 GMT-5)를 따르기 때문에 시차가 발생합니다.
서울과 하와이의 시차는 19시간입니다. 서울은 GMT+9 시간대에 속해 있고, 하와이는 GMT-10 시간대에 속해 있기 때문입니다.
켄버라와 베이징의 시차는 2시간입니다. 켄버라는 GMT+10 시간대에 속하고, 베이징은 GMT+8 시간대에 속하기 때문입니다.


### Stream 활용

In [28]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


In [29]:
# 질의내용
question = "대한민국에 대해서 300자 내외로 최대한 상세히 알려줘"

# 스트리밍으로 답변 출력
response = llm.invoke(question)

대한민국은 동아시아에 위치한 나라로, 수도는 서울이며 인구는 약 5천만 명이 넘는다. 대한민국은 민주공화국으로 정부는 대통령 중심의 체제를 가지고 있으며, 국회는 한국 국회로 불린다. 대한민국은 한반도의 남쪽에 위치하고 있으며 북쪽으로는 북한과 접해 있다.

대한민국은 세계에서 11번째로 경제 규모가 큰 나라로, 주요 산업은 자동차, 전자제품, 선박, 철강 등이 있다. 또한 IT 기술이 발달한 나라로 유명하며, 세계적으로 유명한 기업들이 많이 있다.

문화적으로는 한류가 세계적으로 유명하며, 한국의 음식, 의상, 음악, 영화 등이 전 세계에서 사랑받고 있다. 또한 한국의 전통문화인 한복, 한국 무용, 사물놀이 등도 많은 사람들에게 인기가 있다.

대한민국은 또한 자연환경이 아름다운 나라로, 다양한 산, 강, 바다 등이 있어 관광객들에게 많은 인기를 끌고 있다. 또한 한국의 역사적인 유적지나 문화유산들도 많아 관광지로 유명하다.

### LCEL(LangChain Expression Language)

In [30]:
from langchain_core.output_parsers import StrOutputParser

template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""


prompt = PromptTemplate.from_template(template)


# OpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model="gpt-4-turbo-preview")
# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

영어 회화:
- Hello, I'd like to order, please.
- Sure, what would you like to have?
- I'd like to have the grilled chicken with vegetables, please.
- Anything to drink?
- Yes, I'll have a glass of water, please.
- Alright, your order will be ready soon. Thank you.

한글 해석:
- 안녕하세요, 주문하고 싶습니다.
- 네, 무엇을 드시겠어요?
- 구운 닭고기와 야채를 주세요.
- 음료는요?
- 네, 물 한 잔 주세요.
- 알겠습니다. 곧 준비될 거예요. 감사합니다.


In [31]:
print(chain.invoke({"question": "하와이 고급레스토랑에서 포케 주문"}))

영어 회화:
- Customer: Excuse me, I'd like to try some authentic Hawaiian cuisine. Could you recommend something special from the menu?
- Waiter: Sure! If you're looking for something uniquely Hawaiian, I'd highly recommend our poke bowl. It's a local favorite.
- Customer: That sounds interesting. What exactly is in a poke bowl?
- Waiter: It's a mix of fresh, diced raw fish, usually tuna, that's marinated in soy sauce and sesame oil, then served over rice with a variety of toppings like avocado, seaweed, and cucumber. It's both refreshing and satisfying.
- Customer: That does sound delightful. I'll have the poke bowl, please.
- Waiter: Excellent choice! Would you like any drinks to start with?
- Customer: Yes, a glass of white wine would be great. Thank you.

한글 해석:
- 손님: 실례합니다, 진짜 하와이 음식을 맛보고 싶어요. 메뉴에서 특별한 것을 추천해 주실 수 있나요?
- 웨이터: 물론이죠! 하와이 특유의 것을 찾으신다면 저희 포케볼을 강력히 추천드립니다. 현지인들이 가장 좋아하는 메뉴예요.
- 손님: 흥미롭네요. 포케볼에는 정확히 무엇이 들어가나요?
- 웨이터: 신선하게 다진 생선, 주로 참치가 간장과 참기름에 절여져서, 아보카도, 해초, 오이 같은 다양한 토핑과

In [32]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

# input 딕셔너리에 주제를 'ice cream'으로 설정합니다.
input = {"topic": "양자역학"}

# prompt 객체의 invoke 메서드를 사용하여 input을 전달하고 대화형 프롬프트 값을 생성합니다.
prompt.invoke(input)

StringPromptValue(text='양자역학 에 대해 쉽게 설명해주세요.')

In [33]:
(prompt | model).invoke(input)

AIMessage(content="양자역학은 물질과 에너지의 가장 작은 단위인 '양자'의 세계를 다루는 물리학의 한 분야입니다. 우리가 일상에서 경험하는 고전물리학의 법칙들은 큰 물체에는 잘 적용되지만, 아주 작은 입자들의 세계에서는 다른 현상들이 나타납니다. 양자역학은 이러한 미시세계의 현상을 설명하고 이해하기 위해 개발되었습니다.\n\n양자역학의 주요 개념 몇 가지를 쉽게 설명해보겠습니다:\n\n1. **양자**: 기본적으로 에너지나 물질이 가질 수 있는 가장 작은 단위입니다. 예를 들어, 빛은 무수히 많은 '광자'라는 에너지의 양자들로 구성됩니다.\n\n2. **불확정성 원리**: 하이젠베르크의 불확정성 원리에 따르면, 입자의 위치와 운동량(속도와 질량의 곱)을 동시에 정확하게 알 수 없습니다. 한 쪽을 더 정확하게 알려고 하면, 다른 한 쪽은 더 불확실해집니다. 이는 입자의 세계가 우리가 생각하는 것처럼 결정적이지 않고 확률적인 성질을 가지고 있음을 의미합니다.\n\n3. **파동-입자 이중성**: 양자역학에서는 물질이 입자로도, 파동으로도 행동할 수 있음을 설명합니다. 예를 들어, 빛은 파동의 성질(예: 간섭과 회절)과 입자의 성질(예: 광전효과)을 모두 보입니다. 이는 관찰하는 방식에 따라 물질이 다르게 행동할 수 있음을 의미합니다.\n\n4. **양자 중첩**: 양자역학에서는 입자가 여러 상태의 '중첩' 상태에 있을 수 있습니다. 즉, 하나의 입자가 동시에 여러 위치나 상태에 있을 수 있으며, 관측될 때까지 그 상태가 확정되지 않습니다. 이는 유명한 '슈뢰딩거의 고양이' 실험으로 알려져 있습니다.\n\n5. **양자얽힘**: 두 입자가 양자얽힘 상태에 있을 때, 한 입자의 상태를 변경하면 즉각적으로 다른 입자의 상태도 변경됩니다. 이 현상은 입자들 사이의 거리에 상관없이 일어납니다. 이는 양자역학이 고전물리학과 다른, 매우 비직관적인 성질 중 하나입니다.\n\n양자역학은 우리가 살고 있는 세계를 이해하는 데 매우 중요한 이론이며, 반도체, 레이저,

In [34]:
(prompt | model | output_parser).invoke(input)

'양자역학은 미시세계, 즉 원자나 전자와 같은 아주 작은 입자들이 어떻게 행동하는지를 설명하는 물리학의 한 분야입니다. 우리가 일상생활에서 경험하는 매크로한 세계는 뉴턴의 고전역학 법칙을 따르지만, 미시세계에서는 이 법칙들이 통하지 않아 양자역학이라는 새로운 이론이 필요하게 되었습니다.\n\n양자역학의 몇 가지 중요한 개념을 쉽게 설명해 보겠습니다:\n\n1. **양자**: 양자역학에서는 에너지가 연속적이지 않고 불연속적인 "양자"라는 최소 단위로 존재한다고 봅니다. 예를 들어, 전자가 원자핵 주위를 도는 에너지 레벨은 특정한 단계로만 존재할 수 있으며, 이 사이의 중간 단계는 존재하지 않습니다.\n\n2. **파동-입자 이중성**: 빛이나 전자와 같은 입자는 때에 따라 파동처럼도, 입자처럼도 행동할 수 있습니다. 예를 들어, 빛은 전자기파로 설명될 수 있지만, 빛이 물체에 충돌할 때는 입자처럼 에너지를 전달합니다.\n\n3. **불확정성 원리**: 하이젠베르크의 불확정성 원리에 따르면, 입자의 위치와 운동량(속도와 방향)을 동시에 정확하게 알 수 없습니다. 한 쪽을 더 정확하게 알수록 다른 쪽은 더 불확실해집니다. 이는 양자세계의 기본적인 불확실성을 나타냅니다.\n\n4. **중첩 상태**: 양자역학에서는 입자가 여러 상태의 \'중첩\'으로 존재할 수 있으며, 관측되기 전까지는 모든 가능성을 동시에 가지고 있는 것으로 봅니다. 예를 들어, 슈뢰딩거의 고양이 실험에서 고양이는 살아있는 상태와 죽은 상태가 중첩된 상태로 존재한다고 볼 수 있습니다.\n\n양자역학은 이처럼 비직관적이고 이해하기 어려운 개념들로 가득하지만, 이 이론을 통해 물질의 구조, 화학반응, 고체물리학, 반도체, 레이저 등 현대 과학과 기술의 다양한 분야를 이해할 수 있게 되었습니다.'

In [36]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI()
# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model
chain

ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='{topic} 에 대하여 3문장으로 설명해줘.'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x12f78f9e0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12f5e4350>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [37]:
# 체인의 입력 스키마는 첫 번째 부분인 프롬프트의 입력 스키마입니다.
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [39]:
# 모델의 입력 스키마를 출력합니다.
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'strin

In [40]:
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'name': {'title': 'Name', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}

In [41]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
for s in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(s.content, end="", flush=True)

멀티모달은 여러 가지 형태의 통신 매체를 결합하여 정보를 전달하거나 소통하는 방식을 말합니다. 이는 텍스트, 음성, 이미지, 동영상 등 다양한 매체를 활용하여 사용자에게 다양한 경험을 제공합니다. 멀티모달은 사용자들이 정보를 빠르게 이해하고 상호작용할 수 있는 환경을 조성하여 효과적인 커뮤니케이션을 가능하게 합니다.

In [42]:
# chain 객체의 invoke 메서드를 호출하고, 'ChatGPT'라는 주제로 딕셔너리를 전달합니다.
chain.invoke({"topic": "ChatGPT"})

AIMessage(content='ChatGPT는 인공 지능 챗봇으로, 자연어 처리 기술을 사용하여 대화를 이어나가며 사용자와 상호작용합니다. 사용자의 질문에 대답하거나 대화를 이끌어가며 다양한 주제에 대해 대화할 수 있습니다. ChatGPT는 심층 학습 알고리즘을 기반으로 하여 지속적인 학습을 통해 사용자 경험을 개선하고 더욱 자연스러운 대화를 제공합니다.', response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 24, 'total_tokens': 177}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [43]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

[AIMessage(content='ChatGPT는 OpenAI에서 개발된 자연어 처리 모델로, 대화형 인공지능 채팅을 제공합니다. 사용자의 질문에 대답하거나 대화를 이어나가며 다양한 주제에 대해 대화할 수 있습니다. ChatGPT는 다양한 분야에서 유용하게 활용되며, 자연스러운 대화를 통해 사용자와 상호작용합니다.', response_metadata={'token_usage': {'completion_tokens': 122, 'prompt_tokens': 24, 'total_tokens': 146}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='Instagram은 사진과 동영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상을 시각적으로 나타내고 다른 사람들과 소통할 수 있다. 다양한 필터와 편집 기능을 제공하여 보다 멋진 사진과 동영상을 만들 수 있으며, 해시태그를 통해 관심사나 주제별로 게시물을 찾을 수 있다. 또한 인플루언서나 브랜드들이 제품 홍보나 마케팅을 위해 활발히 활용하는 플랫폼이기도 하다.', response_metadata={'token_usage': {'completion_tokens': 204, 'prompt_tokens': 22, 'total_tokens': 226}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})]

In [44]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

[AIMessage(content='ChatGPT는 인공지능 챗봇으로 자연어 처리 기술을 이용하여 대화를 수행하는 서비스이다. 사용자의 질문이나 요청에 대해 즉각적이고 정확한 대답을 제공하여 소통을 원활하게 도와준다. 다양한 분야에서 활용되며, 고객 서비스, 교육, 엔터테인먼트 등 다양한 산업 분야에 적용될 수 있다.', response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 24, 'total_tokens': 177}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='Instagram은 사진과 동영상을 공유하고 소셜 네트워크를 형성할 수 있는 앱이다. 사용자들은 팔로워들과 소통하며 일상을 공유하고 인스타그램 스토리, 피드, IGTV 등 다양한 기능을 통해 콘텐츠를 공유할 수 있다. 또한 해시태그를 활용하여 관심사나 주제별로 콘텐츠를 찾거나 발견할 수 있다.', response_metadata={'token_usage': {'completion_tokens': 159, 'prompt_tokens': 22, 'total_tokens': 181}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='멀티모달은 여러 가지 다양한 형식의 정보나 자료를 결합하여 제공하는 시스템을 말합니다. 이는 사용자가 보다 풍부한 경험을 얻을 수 있도록 도와줍니다. 예를 들어, 텍스트, 이미지, 음성, 동영상 등을 함께 활용하여 정보를 전달하는 것이 멀티모달의 한 예시입니다.', response_metadata={'token

In [45]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
await chain.ainvoke({"topic": "NVDA"})

AIMessage(content='NVDA는 미국의 기업으로, 주로 시각 장애인을 위한 컴퓨터 소프트웨어와 하드웨어를 제조하고 판매합니다. NVDA의 제품들은 시각 장애인들이 컴퓨터를 보다 쉽게 사용할 수 있도록 도와줍니다. 회사는 전 세계 시각 장애인들에게 혁신적인 기술을 제공하여 더 나은 삶의 질을 제공하고 있습니다.', response_metadata={'token_usage': {'completion_tokens': 154, 'prompt_tokens': 23, 'total_tokens': 177}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [46]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
await chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

[AIMessage(content='YouTube는 동영상 콘텐츠를 시청하고 공유할 수 있는 온라인 동영상 플랫폼이다. 사용자들은 다양한 주제의 동영상을 업로드하고 시청할 수 있으며, 구독과 좋아요 등의 기능을 통해 콘텐츠에 대한 피드백을 제공할 수 있다. 또한, 광고 수익을 얻을 수 있는 방법으로 유명한 크리에이터들도 많이 활동하고 있다.', response_metadata={'token_usage': {'completion_tokens': 167, 'prompt_tokens': 22, 'total_tokens': 189}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='Instagram은 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼이다. 사용자들은 자신의 일상을 담은 콘텐츠를 업로드하고 다른 사람들과 소통할 수 있다. 해시태그를 통해 관심사나 주제에 맞는 콘텐츠를 찾을 수 있다.', response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 22, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content='Facebook은 전세계적으로 가장 인기 있는 소셜 네트워크 서비스 중 하나로, 사용자들이 자신의 생각, 사진, 동영상을 공유하고 소통하는 플랫폼입니다. 또한 광고 및 비즈니스 활동을 위한 도구로도 활발히 활용되고 있습니다. 사용자들은 친구, 가족뿐만 아니라 새로운 인연을 만나고 소통할 수 있는 다양한 기능을 제공합니다.', response_metadata={